# Exploring Burnout

Analyze the performance of our Whittle and Adaptive Policies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
import secrets
from itertools import combinations

In [3]:
from rmab.simulator import run_multi_seed
from rmab.whittle_policies import *
from rmab.baseline_policies import *
from rmab.mcts_policies import *
from rmab.utils import get_save_path, delete_duplicate_results
import rmab.secret as secret
from rmab.database import *
from rmab.fr_dynamics import *
from collections import Counter, defaultdict
import bisect

/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
is_jupyter = 'ipykernel' in sys.modules

## Retrieving Data

In [5]:
db_name = secret.database_name 
username = secret.database_username 
password = secret.database_password 
ip_address = secret.ip_address
port = secret.database_port
connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [6]:
query = "SELECT * FROM RESCUES"
all_rescue_data = run_query(cursor,query)

query = ("SELECT * FROM ADDRESSES")
all_addresses = run_query(cursor,query)

address_id_to_latlon = {}
address_id_to_state = {}
for i in all_addresses:
    address_id_to_state[i['id']] = i['state']
    address_id_to_latlon[i['id']] = (i['latitude'],i['longitude'])

In [7]:
user_id_to_latlon = {}
user_id_to_state = {}
user_id_to_start = {}
user_id_to_end = {}

query = ("SELECT * FROM USERS")
user_data = run_query(cursor,query)

for user in user_data:
    if user['address_id'] != None: 
        user_id_to_latlon[user['id']] = address_id_to_latlon[user['address_id']]
        user_id_to_state[user['id']] = address_id_to_state[user['address_id']]
        user_id_to_start[user['id']] = user['created_at']
        user_id_to_end[user['id']] = user['updated_at']



In [8]:
query = (
    "SELECT * "
    "FROM RESCUES "
    "WHERE PUBLISHED_AT <= CURRENT_DATE "
    "AND USER_ID IS NOT NULL "
)

all_user_published = run_query(cursor,query)

query = (
    "SELECT * FROM donor_locations"
)
data = run_query(cursor,query)
donor_location_to_latlon = {}
for i in data:
    donor_location_to_latlon[i['id']] = address_id_to_latlon[i['address_id']]

query = (
    "SELECT * FROM donations"
)
donation_data = run_query(cursor,query)

donation_id_to_latlon = {}
for i in donation_data:
    donation_id_to_latlon[i['id']] = donor_location_to_latlon[i['donor_location_id']]


In [9]:
query = (
    "SELECT USER_ID, PUBLISHED_AT "
    "FROM RESCUES "
    "WHERE PUBLISHED_AT <= CURRENT_DATE "
    "AND USER_ID IS NOT NULL "
)

all_user_published = run_query(cursor,query)

data_by_user = {}
for i in all_user_published:
    user_id = i['user_id']
    published_at = i['published_at']

    if user_id not in data_by_user:
        data_by_user[user_id] = []

    data_by_user[user_id].append(published_at)

num_rescues_to_user_id = {}
for i in data_by_user:
    if len(data_by_user[i]) not in num_rescues_to_user_id:
        num_rescues_to_user_id[len(data_by_user[i])] = []
    num_rescues_to_user_id[len(data_by_user[i])].append(i)

rescue_to_latlon = {}
rescue_to_time = {}
for i in all_rescue_data:
    if i['published_at'] != None and donation_id_to_latlon[i['donation_id']] != None and donation_id_to_latlon[i['donation_id']][0] != None:
        rescue_to_latlon[i['id']] = donation_id_to_latlon[i['donation_id']]
        rescue_to_latlon[i['id']] = (float(rescue_to_latlon[i['id']][0]),float(rescue_to_latlon[i['id']][1]))
        rescue_to_time[i['id']] = i['published_at']


In [10]:
def count_notifications_per_rescue(notifications, trips):
    rescue_counts = []  
    rescue_pointer = 0
    
    for trip_date in trips:
        while rescue_pointer < len(notifications) and notifications[rescue_pointer] < trip_date - timedelta(days=7):
            rescue_pointer += 1
        
        count = 0
        temp_pointer = rescue_pointer
        
        while temp_pointer < len(notifications) and notifications[temp_pointer] < trip_date:
            count += 1
            temp_pointer += 1
        
        rescue_counts.append(count)
    
    return rescue_counts



In [11]:
def count_notifications_per_week(notifications, trips):
    weekly_rescues = defaultdict(int)
    weekly_trips = defaultdict(bool)

    for rescue in notifications:
        week = rescue.isocalendar()[:2]
        weekly_rescues[week] += 1

    for trip in trips:
        week = trip.isocalendar()[:2]
        weekly_trips[week] = True

    result = []
    for week in sorted(weekly_rescues.keys()):
        result.append({
            'week_start': week,
            'num_notifications': weekly_rescues[week],
            'trip_completed': weekly_trips.get(week, False)
        })

    return result


In [12]:
def num_notifications_last_week(user_id):
    user_location = user_id_to_latlon[user_id]
    if user_location[0] == None:
        return [], 0
    user_location = (float(user_location[0]),float(user_location[1]))
    user_end = user_id_to_end[user_id]
    user_start = user_id_to_start[user_id]

    notifications = [i for i in rescue_to_time if user_start <= rescue_to_time[i] and rescue_to_time[i] <= user_end]
    notifications = [i for i in notifications if haversine(user_location[0],user_location[1],rescue_to_latlon[i][0],rescue_to_latlon[i][1]) < 5]
    notification_times = [rescue_to_time[i] for i in notifications]
    
    notification_times = sorted(notification_times)
    trip_dates = sorted(data_by_user[user_id])
    notifications_per_rescue = count_notifications_per_rescue(notification_times,trip_dates)
    notifications_per_week = count_notifications_per_week(notification_times,trip_dates)
    return notifications_per_rescue, notifications_per_week

In [13]:
def get_interval_num(intervals,num):
    i = 0
    while i<len(intervals):
        if num<intervals[i]:
            return i 
        i+=1
    return len(intervals)

In [14]:
def get_all_notification_data(num_trips):
    notifications_by_week = []
    notifications_by_rescue = []

    for i in num_rescues_to_user_id[num_trips]:
        if i in user_id_to_latlon:
            notifications_per_rescue_user, notifications_per_week_user = num_notifications_last_week(i)
            
            notifications_by_week.append(notifications_per_week_user)
            notifications_by_rescue += notifications_per_rescue_user
    return notifications_by_week, notifications_by_rescue

In [18]:
def compute_recovery_rates(notifications_by_week,intervals):
    num_trips_completed_after_by_notifications = {}
    for i in intervals:
        num_trips_completed_after_by_notifications[i] = [0,0]

    for i in notifications_by_week:
        for user in notifications_by_week:
            if user != 0:
                for idx,row in enumerate(user[:-1]):
                    num_notifications = intervals[get_interval_num(intervals,row['num_notifications'])]
                    num_trips_completed_after_by_notifications[num_notifications][int(user[idx+1]['trip_completed'])] += 1
                if len(user)>0:        
                    num_notifications = intervals[get_interval_num(intervals,user[-1]['num_notifications'])]
                    num_trips_completed_after_by_notifications[num_notifications][0] += 1
    for i in num_trips_completed_after_by_notifications:
        num_trips_completed_after_by_notifications[i] = num_trips_completed_after_by_notifications[i][1]/(num_trips_completed_after_by_notifications[i][0]+num_trips_completed_after_by_notifications[i][1])

    return num_trips_completed_after_by_notifications

In [50]:
def compute_weekly_engagement(notifications_by_week,intervals):
    total_weeks_by_situation = np.zeros((2,len(intervals)))
    engagement_rate_after = np.zeros((2,len(intervals)))
    
    for i in notifications_by_week:
        for user in notifications_by_week:
            if user != 0:
                for idx,row in enumerate(user[:-1]):
                    interval_num = get_interval_num(intervals,row['num_notifications'])
                    engaged = int(user[idx]['trip_completed'])
                    next_engaged = int(user[idx+1]['trip_completed'])

                    total_weeks_by_situation[engaged,interval_num] += 1
                    if next_engaged == 1:
                        engagement_rate_after[engaged,interval_num] += 1

    eps = 0.00001
    engagement_rate_after += eps/2
    total_weeks_by_situation += eps 
    transition_matrix = np.zeros((2,len(intervals),2))
    transition_matrix[:,:,1] = engagement_rate_after/total_weeks_by_situation
    transition_matrix[:,:,0] = 1-transition_matrix[:,:,1]

    return transition_matrix

In [17]:
def compute_burnout_curve(notifications_by_rescue):
    count_freq = Counter(notifications_by_rescue)

    unique_counts = sorted(count_freq.keys())

    total_counts = len(notifications_by_rescue)
    cdf_values = []
    for k in unique_counts:
        fraction = sum(freq for count, freq in count_freq.items() if count >= k) / total_counts
        cdf_values.append(fraction)
    return unique_counts, cdf_values 

def compute_notification_burnout_rates(notifications_by_rescue,intervals):
    unique_counts, cdf_values = compute_burnout_curve(notifications_by_rescue)
    burnout_values = []

    for idx,i in enumerate(intervals):
        loc = bisect.bisect_left(unique_counts,i)
        if loc == len(unique_counts):
            loc = len(unique_counts)-1
        if idx == 0:
            burnout_values.append(1-cdf_values[loc]**(1/unique_counts[loc]))
        else:
            loc_before = bisect.bisect_left(unique_counts,intervals[idx-1])
            burnout_values.append(1-(cdf_values[loc]/cdf_values[loc_before])**(1/(unique_counts[loc]-unique_counts[loc_before])))
    return burnout_values

In [16]:
intervals = [10,50,100,1000]

In [17]:
all_notifications_by_rescue = json.load(open("../../results/food_rescue/notifications_by_rescue.json"))
all_notifications_by_week = json.load(open("../../results/food_rescue/notifications_by_week.json"))

In [57]:
all_weekly_transitions = {}

for i in all_notifications_by_rescue:
    print("On {}".format(i))
    try:
        all_weekly_transitions[i] = compute_weekly_engagement(all_notifications_by_week[i],intervals).tolist()
        last_transition = all_weekly_transitions[i]

    except:
        all_weekly_transitions[i] = last_transition



On 3
On 4
On 5
On 6
On 7
On 8
On 9
On 10
On 11
On 12
On 13
On 14
On 15
On 16
On 17
On 18
On 19
On 20
On 21
On 22
On 23
On 24
On 25
On 26
On 27
On 28
On 29
On 30
On 31
On 32
On 33
On 34
On 35
On 36
On 37
On 38
On 39
On 40
On 41
On 42
On 43
On 44
On 45
On 46
On 47
On 48
On 49
On 50
On 51
On 52
On 53
On 54
On 55
On 56
On 57
On 58
On 59
On 60
On 61
On 62
On 63
On 64
On 65
On 66
On 67
On 68
On 69
On 70
On 71
On 72
On 73
On 74
On 75
On 76
On 77
On 78
On 79
On 80
On 81
On 82
On 83
On 84
On 85
On 86
On 87
On 88
On 89
On 90
On 91
On 92
On 93
On 94
On 95
On 96
On 97
On 98
On 99
On 100
On 101
On 102
On 103
On 104
On 105
On 106
On 107
On 108
On 109
On 110
On 111
On 112
On 113
On 114
On 115
On 116
On 117
On 118
On 119
On 120
On 121
On 122
On 123
On 124
On 125
On 126
On 127
On 128
On 129
On 130
On 131
On 132
On 133
On 134
On 135
On 136
On 137
On 138
On 139
On 140
On 141
On 142
On 143
On 144
On 145
On 146
On 147
On 148
On 150
On 151
On 152
On 153
On 154
On 155
On 156
On 157
On 158
On 159
On 160
On 16

In [58]:
json.dump(all_weekly_transitions,open('../../results/food_rescue/all_weekly_transitions.json','w'))

In [24]:
all_burnout_rates = {}
last_burnout_rate = {}

for i in all_notifications_by_rescue:
    try:
        all_burnout_rates[i] = compute_burnout_rates(all_notifications_by_rescue[i],intervals)
        last_burnout_rate = all_burnout_rates[i]
    except:
        all_burnout_rates[i] = last_burnout_rate
json.dump(all_burnout_rates,open("../../results/food_rescue/all_burnout_rates.json","w"))

In [22]:
all_recovery_rates = {}
last_recovery_rate = {}

for i in all_notifications_by_week:
    try:
        all_recovery_rates[i] = compute_recovery_rates(all_notifications_by_week[i],intervals)
        last_recovery_rate = all_recovery_rates[i]
    except:
        all_recovery_rates[i] = last_recovery_rate
json.dump(all_recovery_rates,open("../../results/food_rescue/all_recovery_rates.json","w"))

In [25]:
all_recovery_rates['25']

{10: 0.32020997375328086,
 50: 0.22361623616236162,
 100: 0.2476340694006309,
 1000: 0.17581047381546136}

In [23]:
notifications_by_week, notifications_by_rescue = get_all_notification_data(100)

In [34]:
compute_recovery_rates(notifications_by_week,intervals)

{10: 0.5542168674698795,
 50: 0.8235294117647058,
 100: 0.5,
 1000: 0.5271317829457365}

In [41]:
compute_burnout_rates(notifications_by_rescue,intervals)

[0.04165358689352583,
 0.0054762080193774265,
 0.00010151339114172764,
 0.007627463182702687]

In [46]:
all_notifications_by_week = {}
all_notifications_by_rescue = {}

for num_trips in sorted(num_rescues_to_user_id.keys()):
    print("On num trips {}".format(num_trips))
    if num_trips >= 3 and len(num_rescues_to_user_id[num_trips])>0:
        notifications_by_week, notifications_by_rescue = get_all_notification_data(num_trips)
        all_notifications_by_week[num_trips] = notifications_by_week
        all_notifications_by_rescue[num_trips] = notifications_by_rescue

On num trips 1
On num trips 2
On num trips 3
On num trips 4
On num trips 5
On num trips 6
On num trips 7
On num trips 8
On num trips 9
On num trips 10
On num trips 11
On num trips 12
On num trips 13
On num trips 14
On num trips 15
On num trips 16
On num trips 17
On num trips 18
On num trips 19
On num trips 20
On num trips 21
On num trips 22
On num trips 23
On num trips 24
On num trips 25
On num trips 26
On num trips 27
On num trips 28
On num trips 29
On num trips 30
On num trips 31
On num trips 32
On num trips 33
On num trips 34
On num trips 35
On num trips 36
On num trips 37
On num trips 38
On num trips 39
On num trips 40
On num trips 41
On num trips 42
On num trips 43
On num trips 44
On num trips 45
On num trips 46
On num trips 47
On num trips 48
On num trips 49
On num trips 50
On num trips 51
On num trips 52
On num trips 53
On num trips 54
On num trips 55
On num trips 56
On num trips 57
On num trips 58
On num trips 59
On num trips 60
On num trips 61
On num trips 62
On num trips 63
O

In [47]:
json.dump(all_notifications_by_week,open('../../results/food_rescue/notifications_by_week.json','w'))
json.dump(all_notifications_by_rescue,open('../../results/food_rescue/notifications_by_rescue.json','w'))